In [ ]:
# %run ./model_end_events_for_outages_METHODS.ipynb

In [ ]:
from importlib import reload
#reload(Utilities)
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

import sys, os
import re
import string
from pathlib import Path
import json
import pickle
import joblib

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version
import copy
from functools import reduce

import itertools

import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.colors as mcolors
import matplotlib.cm as cm #e.g. for cmap=cm.jet
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
from MeterPremise import MeterPremise
from EEMSP import EEMSP
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from MECPODf import MECPODf
from MECPOAn import MECPOAn
from MECPOCollection import MECPOCollection
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
from OutageModeler import OutageModeler
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_General
import Plot_Box_sns
import Plot_Hist
import Plot_Bar
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSingleSlicer as DFSingleSlicer
from CustomJSON import CustomEncoder, CustomWriter

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GroupShuffleSplit, GridSearchCV, RandomizedSearchCV, cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve, roc_curve

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn import preprocessing

import tensorflow as tf
from tensorflow import keras

import scipy

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from KMeansWeightedConstrained import KMeansWeightedConstrained

In [ ]:
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

# ---------------------------------------------------------------------------------------------------

In [ ]:
trsf_pole_df_outg = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20231201\20230401_20230930\Outages\trsf_location_info_df.pkl')
trsf_pole_df_otbl = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20231201\20230401_20230930\OutgXfmrBaseline\trsf_location_info_df.pkl')
trsf_pole_df_prbl = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20231201\20230401_20230930\PristineBaseline\trsf_location_info_df.pkl')
#-------------------------
# trsf_pole_df = pd.concat([trsf_pole_df_outg, trsf_pole_df_otbl, trsf_pole_df_prbl])
trsf_pole_df = trsf_pole_df_outg
trsf_pole_df = trsf_pole_df.drop_duplicates()

In [ ]:
trsf_pole_df

In [ ]:
zip_pop_df = pd.read_csv(r'C:\Users\s346557\Downloads\DECENNIALDHC2020.P1_2024-04-01T151417\DECENNIALDHC2020.P1-Data.csv')
zip_pop_df = zip_pop_df.iloc[1:][['NAME', 'P1_001N']].copy()
zip_pop_df['zip'] = zip_pop_df['NAME'].apply(lambda x: Utilities.find_and_replace_in_string(x, '.* (\d*)', r'\1'))
zip_pop_df = zip_pop_df.rename(columns={'P1_001N':'population'}).drop(columns=['NAME'])
zip_pop_df = Utilities_df.convert_col_type(df=zip_pop_df, column='population', to_type='int')
zip_pop_df['zip_as_int'] = zip_pop_df['zip'].copy()
zip_pop_df = Utilities_df.convert_col_type(df=zip_pop_df, column='zip_as_int', to_type='int')
#-------------------------
zip_loc_df = pd.read_pickle(r'C:\Users\s346557\Downloads\2023_Gaz_zcta_national\zip_lat_long_df.pkl')
#-------------------------
zip_pop_df = pd.merge(
    zip_pop_df, 
    zip_loc_df, 
    how='left', 
    left_on=['zip'], 
    right_on=['zip']
)

In [ ]:
zips_not_found = list(set(trsf_pole_df['zip']).difference(set(zip_pop_df['zip'])))
print(f'#zips in trsf_pole_df not found: {len(zips_not_found)}')

In [ ]:
zips_not_found

# I want them combined with (in order or importance)
1. other zips of approximately equal size
2. other zips nearby (simply judged by having similar zip code numbers)

In [ ]:
print(zip_pop_df['zip'].nunique())
print(trsf_pole_df['zip'].nunique())

In [ ]:
# zip_pop_df = zip_pop_df[zip_pop_df['zip'].isin(trsf_pole_df['zip'].unique())].copy()
zip_pop_df = zip_pop_df[zip_pop_df['zip'].isin(trsf_pole_df_outg['zip'].unique())].copy()

In [ ]:
zip_pop_df

In [ ]:
clf_4 = KMeansWeightedConstrained(
    n_clusters=50,
    size_min=50000,
    size_max=200000,
    random_state=0
)
labels_4 = clf_4.fit_predict(zip_pop_df[['zip_as_int', 'lat', 'long']].to_numpy(), zip_pop_df['population'].to_numpy())
#-------------------------
assert(zip_pop_df.shape[0]==len(labels_4))
zip_pop_df['cluster_4'] = labels_4
cluster_pops_4 = zip_pop_df.groupby(['cluster_4'])['population'].sum()

In [ ]:
cluster_pops_4

In [ ]:
print(cluster_pops_4.min())
print(cluster_pops_4.max())
print()

In [ ]:
clf_5 = KMeansWeightedConstrained(
    n_clusters=50,
    size_min=50000,
    size_max=200000,
    random_state=0
)
labels_5 = clf_5.fit_predict(zip_pop_df[['lat', 'long']].to_numpy(), zip_pop_df['population'].to_numpy())
#-------------------------
assert(zip_pop_df.shape[0]==len(labels_5))
zip_pop_df['cluster_5'] = labels_5
cluster_pops_5= zip_pop_df.groupby(['cluster_5'])['population'].sum()

In [ ]:
cluster_pops_5

In [ ]:
print(cluster_pops_4.min())
print(cluster_pops_4.max())
print()
print(cluster_pops_5.min())
print(cluster_pops_5.max())
print()

In [ ]:
us_shp_path = r'C:\Users\s346557\Downloads\tl_2023_us_state\tl_2023_us_state.shp'
us_df = gpd.read_file(us_shp_path)
us_df = us_df.to_crs("EPSG:4326")
#-----
non_continental = ['HI','VI','MP','GU','AK','AS','PR']
us49 = us_df[~us_df['STUSPS'].isin(non_continental)].copy()
us49.plot()

In [ ]:
us_zcta_path = r'C:\Users\s346557\Downloads\tl_2023_us_zcta520\tl_2023_us_zcta520.shp'
us_zcta_df = gpd.read_file(us_zcta_path)
us_zcta_df = us_zcta_df.to_crs("EPSG:4326")
#-----
us_zcta_df = Utilities_df.convert_col_type(df=us_zcta_df, column='INTPTLAT20', to_type=float)
us_zcta_df = Utilities_df.convert_col_type(df=us_zcta_df, column='INTPTLON20', to_type=float)


# Below, need to 
Include more assertion restrictions regarding zip_pop_df, us_zcta_df, and states_shp_df
I need a better understanding of us_zcta_df, and states_shp_df before this is possible

In [ ]:
import random
import geopandas as gpd
def draw_zip_groups(
    zip_pop_df, 
    us_zcta_df, 
    states_shp_df, 
    n_colors           = 25, 
    draw_hatches       = True, 
    dense_only_hatches = True, 
    zip_col            = 'zip', 
    cluster_col        = 'cluster_5', 
    geoid20_col        = 'GEOID20'
):
    r"""
    states_shp_df:
        Should be a GeoPandas DF take from shape file (I'm super new to GeoPandas, so I don't have a great grasp on vernacular).
        e.g., from development, I used (the tl_2023_us_state.shp was downloaded from US Census Bureau or whoever)
            us_shp_path = r'C:\Users\s346557\Downloads\tl_2023_us_state\tl_2023_us_state.shp'
            us_df = gpd.read_file(us_shp_path)
            us_df = us_df.to_crs("EPSG:4326")
            states_shp_df = us_df[us_df['STUSPS']=='OH']

    draw_hatches:
        Toss up whether True or False should be default.
        False is simpler in appearance, but colors most likely repeat, so, without the hatches it is
          sometimes difficult to differentiate groups (although, groups are usually clustered in space, so, 
          although group colors may be repeated, the two clusters should typically be distinguishable in space)
        True is a little more complicated in appearance, but groups should be more distinguishable as two groups sharing
          the same color/hatch combo is much more unlikely than two groups sharing the same color
    """
    #-------------------------
    colors  = Plot_General.get_standard_colors( n_colors   = n_colors)
    hatches = Plot_General.get_standard_hatches(dense_only = dense_only_hatches)
    #-----
    if zip_pop_df[cluster_col].nunique() > len(colors):
        colors  = random.choices(colors,  k=zip_pop_df[cluster_col].nunique())
    hatches = random.choices(hatches, k=zip_pop_df[cluster_col].nunique())
    #-------------------------
    fig, ax = Plot_General.default_subplots()
    states_shp_df.boundary.plot(ax=ax)
    #-----
    for i,clstr_i in enumerate(natsorted(zip_pop_df[cluster_col].unique())):
        zips_i = zip_pop_df[zip_pop_df[cluster_col]==clstr_i][zip_col].unique()
        df_i   = us_zcta_df[us_zcta_df[geoid20_col].isin(zips_i)]
        color_i = colors[i]
        hatch_i = hatches[i]
        if draw_hatches:
            # When hatches are included for this plot, I find it best to have the hatches in white and boundaries in black.
            # This separation requires the use of df_i.plot and df_i.boundary.plot
            df_i.plot(ax=ax, color=color_i, hatch=hatch_i, edgecolor='white', linewidth=0.5)
            df_i.boundary.plot(ax=ax, color='black', linewidth=0.5)
        else:
            df_i.plot(ax=ax, color=color_i, hatch=None, edgecolor='white', linewidth=0.5)
    #-------------------------
    return fig, ax

In [ ]:
fig, ax = draw_zip_groups(
    zip_pop_df, 
    us_zcta_df, 
    states_shp_df      = us_df[us_df['STUSPS']=='OH'], 
    draw_hatches       = True, 
)

In [ ]:
fig, ax = draw_zip_groups(
    zip_pop_df, 
    us_zcta_df, 
    states_shp_df      = us_df[us_df['STUSPS']=='OH'], 
    draw_hatches       = False, 
)

In [ ]:
zip_pop_df

In [ ]:
zip_pop_df.groupby(['cluster_5']).size()

In [ ]:
zip_pop_df.groupby(['cluster_5'])['zip'].agg(list)

In [ ]:
zip_pop_df.groupby(['cluster_5'])['population'].sum()

In [ ]:
zip_pop_df.groupby(['cluster_5'])['zip'].agg(list)[2]

In [ ]:
# zip_pop_df.to_pickle(r'C:\Users\s346557\Downloads\zip_pop_df.pkl')